In [ ]:
#!/usr/bin/env python
# coding: utf-8

In[ ]:

In[ ]:

### Predict outcome variable Var using 10 Resting State Networks fed to the Nearest Centroid Algorithm 

#### Data Loading and Organization

In[151]:

Load in required libaries 

In[152]:

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os 
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import confusion_matrix, roc_auc_score,plot_confusion_matrix, balanced_accuracy_score
from sklearn.neighbors import NearestCentroid
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import sem, iqr, norm

In[153]:

change current directory to where you have stored the data

In[154]:

set current working directory

In [ ]:
path_to_data = os.getcwd()
os.chdir(path_to_data)    

In[155]:

network names

In[156]:

In [ ]:
networkLabs = ["medial visual", "occipital pole visual", "lateral visual", "default mode", "cerebellar", "sensorimotor", "auditory",  "executive control", "right frontoparietal", "left frontoparietal"]

In[157]:

get index for networks used in prediction    

In[158]:

In [ ]:
RSNidx = np.linspace(0,len(networkLabs) -1,len(networkLabs)).astype('int')

In[159]:

load in datafile with resting state spatial correlations

In [ ]:
dfML = pd.read_csv("SpatialCorrelation.csv")
# load in outcome file
dfOutcome = pd.read_csv("ClinicalInformation.csv")

In[160]:

drop patients with unacceptably large motion artifacts from the dataset

In[161]:

In [ ]:
bSubs = [8,18]; 
dfML = dfML[(dfML["Subjects"] != bSubs[0]) & (dfML["Subjects"] != bSubs[1])]
dfOutcome = dfOutcome[(dfOutcome["Patient"] != bSubs[0]) & (dfOutcome["Patient"] != bSubs[1])].reset_index()
# resort subject index 
dfML['Subjects'] = np.arange(0,25)

In[162]:

optionally drop networks from analysis

In[163]:

In [ ]:
keepNets = ["Subjects","1","2","3","4","5","6","7","8","9","10"]
netIdx = [int(num)-1 for num in keepNets[1:]]
networkLabs = np.array(networkLabs)[netIdx]
print(networkLabs)

In[164]:

make a numpy array from the networks kept in the analysis

In [ ]:
X = np.array(dfML.loc[:,keepNets])
X = X[:,1:]
# apply fisher transform to new patients (who were concatenated from a new dataset)
X[18:,:] = np.arctanh(X[18:,:])

In[165]:

scale features to minmax

In [ ]:
scaler = MinMaxScaler()
# apply min max scaler
X = scaler.fit_transform(X)

In[166]:

sort behavioural data  

In [ ]:
dfOutcome["AgeNew"] = dfOutcome["Age"]
dfOutcome['RCNew']  = dfOutcome['GOS'] > 2
dfOutcome['GOSNew'] = dfOutcome["GOS"] > 3
dfOutcome["FinalGOSNew"] = dfOutcome["Final GOS"] > 3
dfOutcome['SexNew'] = dfOutcome["Sex"] == "M"
dfOutcome["TOSNew"] = np.log10(dfOutcome['Time of Scan post-ictus'])
dfOutcome["GCSNew"] = dfOutcome["GCS Total"].fillna(value = np.mean(dfOutcome["GCS Total"])).values 
dfOutcome["GCSENew"] = dfOutcome["GCS_E"].fillna(value = np.mean(dfOutcome["GCS_E"])).values 
dfOutcome["GCSMNew"] = dfOutcome["GCS_M"].fillna(value = np.mean(dfOutcome["GCS_M"])).values 
#dfOutcome["SedationNew"] = dfOutcome["Sedation"].astype("int")
#dfOutcome["SedationNew"] = dfOutcome["SedationNew"].astype("boolean")

#### Initalize paramaters and storage for the Nearest Centroid Model

In[167]:

In [ ]:
nIter = 1000
test_size = 0.30
np.random.seed(59)
Var = "GOSNew" # which variable do we want to predict based on resting state connectivity 
Yout = np.array(dfOutcome[Var]) # outcome variable
Xact = X; 
Xo = scaler.inverse_transform(X) # use original spatial correlation values for plotting

In[168]:

initalize machine learning parameters and results df

In [ ]:
Pred = [];
nSub = Xact.shape[0]; ts = int(test_size * nSub) # test size
# results dataframe
pR = pd.DataFrame()
pR['Subject'] = list(range(0,nSub)) # patient index
pR['Results'] = np.zeros(nSub) # whether 0 or 1
pR['nIter'] = np.zeros(nSub) # number of iterations patient was involved in testing

In[169]:

create null columns in df to store chance level predictions and features

In [ ]:
pR['ResultsNull'] = np.zeros(nSub)   
PredNull = [];

In[170]:

fix the leave participant out index for reproducibility

In [ ]:
randIdx = np.zeros((nIter,nSub)) * np.nan;
subList = np.array(range(0,nSub))
for nn in range(0,nIter):
    randIdx[nn,:] = np.random.permutation(subList)

In [ ]:
nFold = 3; ResultsNull = np.ones((nIter,nSub)) * np.nan
# #### Train the null model

In[171]:

In [ ]:
tROCNull = np.zeros((nIter,nFold)); CentroidsNull = np.zeros((nIter,nFold,2,10))
for kk in range(0,nIter):
    # initalize random seed
    # set train test indicies
    #ridx = randIdx[kk].astype("int")
    # permute test array
    yR = np.random.permutation(Yout) # only difference between null model and regular model 
    # gather train and test sets based on ridx
    # run through a couple folds 
    kf = StratifiedKFold(n_splits = nFold, random_state=(59+kk), shuffle = True)
    kcount = 0
    for train_index, test_index in kf.split(Xact, yR):
         X_train, X_test = Xact[train_index], Xact[test_index]
         y_train, y_test = yR[train_index], yR[test_index]
         # scambled y_test
         y_test = np.random.permutation(y_test)
         clf = NearestCentroid(metric = "euclidean")
         clf.fit(X_train,y_train)
         # store group predictions
         predictionsn = clf.predict(X_test)
         PredNull.append(predictionsn)
         # store individual predictions
         ResultsNull[kk,test_index] = predictionsn
         # store feature weights
         CentroidsNull[kk,kcount,:,:] = clf.centroids_ 
         tROCNull[kk,kcount] = balanced_accuracy_score(y_test,predictionsn)
         kcount += 1
             
# In[172]:

compute label confidence and error of the null distribution

In [ ]:
pR['ResultsNull'] = np.mean(ResultsNull,axis = 0)
dfOutcome['LabelConfNull'] = pR['ResultsNull']
# point the confidence in the direction of the actual label
### NOTE large confidence values means incorrect with high confidence
### low confidence mean correct with high confidence
dfOutcome["LabelConfNull"] = (dfOutcome["LabelConfNull"] - dfOutcome[Var]).abs()

In[173]:

find the parameters of the null distribution

In [ ]:
nMeanS = dfOutcome.groupby(Var).mean()["LabelConfNull"] # mean between the groups
nErrorS = dfOutcome.groupby(Var).std()["LabelConfNull"] # standard deviation between the groups
nS = dfOutcome.groupby(Var).count()["LabelConfNull"] # number of patients in each group
nErrorSE = nErrorS * 1.96 # CI 
nMean = dfOutcome["LabelConfNull"].mean(); nError = dfOutcome["LabelConfNull"].std() # mean and std of null
nCI = nError * 1.96 # CI of null

#### Plot the Null Results

In[174]:

In [ ]:
from sklearn.datasets import make_blobs
import colorcet as cc

In[175]:

In [ ]:
sns.set_theme(style = "white", rc={'figure.figsize': (3.1,3.1),"xtick.labelsize": 12,             'xtick.alignment':'center',"ytick.labelsize": 12,'font.family':'Arial','legend.fontsize':12,'figure.dpi':600,             "axes.labelsize":12,"figure.facecolor":"w","axes.spines.right": False,"axes.spines.left": False,                "axes.spines.top": False,"axes.spines.bottom": False})
    
blobs, labels = make_blobs(n_samples=1000, centers=25, center_box=(-100, 100))
palette = sns.color_palette(cc.glasbey, n_colors=nSub)
pal = sns.color_palette("hls",16)    

In[176]:

In [ ]:
fig, ax = plt.subplots(1,1)
sns.swarmplot(data = dfOutcome, x = Var, y = 'LabelConfNull',              palette = [pal[0],pal[-5]] ,s=7,ax = ax)
x1 = ax.get_xbound()[0]; x2 = ax.get_xbound()[1]
# draw indeterminant region if null is combined based on both outcomes
ax.axhline(y = nMean, xmin = ax.get_xbound()[0], xmax = ax.get_xbound()[1]) 
# plot the confidence interval of the null distribution
ax.fill_between([ax.get_xbound()[0],ax.get_xbound()[1]],nMean - nCI, nMean + nCI, alpha = 0.2)
ax.fill_between([ax.get_xbound()[0],ax.get_xbound()[1]],-0.05, nMean - nCI, alpha = 0.2, color="green")
ax.fill_between([ax.get_xbound()[0],ax.get_xbound()[1]],nMean + nCI,1.05, alpha = 0.2, color="red")
plt.xlabel(Var)
plt.ylim([0,1])
ax.set_xlabel("Outcome")
ax.set_ylim([-0.05,1.05])
ax.set_yticks([0,0.25,0.5,0.75,1])
ax.set_yticklabels(["100%","75%","50%","75%","100%"]) # change tick labels to make the results more interpretable
ax.set_ylabel("Confidence")
ax.set_xticklabels(["Poor","Good"])
plt.show()

plot null group results

In [ ]:
fig, ax = plt.subplots(1,1)
plt.hist(np.mean(tROCNull,axis = 1),color = pal[9])
ax.set_ylabel("Frequency"); ax.set_xlabel("Null Balanced Accuracy")

#### Train the actual model

In[181]:

In [ ]:
Results = np.ones((nIter,nSub)) * np.nan

In[182]:

In [ ]:
tROC = np.zeros((nIter,nFold))
CentroidsReal = np.zeros((nIter,nFold,2,10))
for kk in range(0,nIter):
    # gather train and test sets  
    # run through a couple folds 
    kf = StratifiedKFold(n_splits = nFold, random_state=(59 + kk), shuffle = True)
    kcount = 0
    for train_index, test_index in kf.split(Xact, yR):
         #print(train_index.shape); print(test_index.shape); print(kk)
         X_train, X_test = Xact[train_index], Xact[test_index]
         y_train, y_test = Yout[train_index], Yout[test_index]
         clf = NearestCentroid(metric = "euclidean")
         clf.fit(X_train,y_train)
         # store group predictions
         predictions = clf.predict(X_test)
         Pred.append(predictions)
         # store individual predictions
         Results[kk,test_index] = predictions
         # store feature weights
         CentroidsReal[kk,kcount,:,:] = clf.centroids_
         tROC[kk,kcount] = balanced_accuracy_score(y_test,predictions)    
         kcount += 1

In[183]:

store CI and descriptives of group results

In [ ]:
mtROC = np.mean(tROC); medtROC = np.median(tROC)
stROC = sem(tROC); iqrtROC = iqr(tROC)
prtROC = np.percentile(tROC,[25,75])

In[184]:

tore actual label confidence

In [ ]:
dfOutcome['LabelConf'] = np.mean(Results,axis = 0)
# point the confidence in the direction of the actual label
### NOTE large confidence values means incorrect with high confidence
### low confidence mean correct with high confidence
dfOutcome["LabelConf"] = (dfOutcome["LabelConf"] - dfOutcome[Var]).abs() 

In[185]:

In [ ]:
fig,ax = plt.subplots()
sns.swarmplot(data = dfOutcome, x = Var, y = 'LabelConf', palette = [pal[0],pal[-5]],s=5,ax = ax)
x1 = ax.get_xbound()[0]; x2 = ax.get_xbound()[1]
ax.axhline(y = nMean, xmin = ax.get_xbound()[0], xmax = ax.get_xbound()[1]) 
# draw indeterminant region if null combined for both outcome groups
ax.fill_between([ax.get_xbound()[0],ax.get_xbound()[1]],nMean - nCI, nMean + nCI, alpha = 0.2)
ax.fill_between([ax.get_xbound()[0],ax.get_xbound()[1]],-0.05, nMean - nCI, alpha = 0.2, color="green")
ax.fill_between([ax.get_xbound()[0],ax.get_xbound()[1]],nMean + nCI,1.05, alpha = 0.2, color="red")
sns.despine(top=True, right=True, left=True, bottom=True)
ax.set_xlabel("Outcome")
ax.set_ylim([-0.05,1.05])
ax.set_yticks([0,0.25,0.5,0.75,1])
ax.set_yticklabels(["100%","75%","50%","75%","100%"]) # make the ylabel more readable
ax.set_ylabel("Confidence")
ax.set_xticklabels(["Poor","Good"])

In[186]:

plot without change in axis for context

In [ ]:
g = sns.catplot(data = dfOutcome, x = Var, y = "LabelConf", palette = "Paired",s=10,kind = "swarm")
plt.axhline(y = nMean, xmin = g.ax.get_xbound()[0], xmax = g.ax.get_xbound()[1]) 
#plot the confidence interval of the null distribution
plt.fill_between([g.ax.get_xbound()[0],g.ax.get_xbound()[1]],nMean - nError, nMean + nError, alpha = 0.2)
plt.fill_between([g.ax.get_xbound()[0],g.ax.get_xbound()[1]],-0.05, nMean - nError, alpha = 0.2, color="green")
plt.fill_between([g.ax.get_xbound()[0],g.ax.get_xbound()[1]],nMean + nError,1.05, alpha = 0.2, color="red")
plt.xlabel(Var)
sns.despine(top=True, right=True, left=True, bottom=True)
plt.ylabel("Label Error")
plt.xticks([0,1],["No","Yes"])
plt.ylim([-0.05,1.05])
plt.show()

In[187]:

lot when null is produced by the different outcome groups

In [ ]:
fig,ax = plt.subplots(nrows = 1, ncols = 1)
g = sns.swarmplot(data = dfOutcome, x = Var, y = "LabelConf",palette = [pal[0],pal[-5]],s=5,ax = ax)
#g = sns.swarmplot(data = dfOutcome, x = Var, y = "LabelConf", hue = "StudyID",s=5,ax = ax)
# plot the confidence interval of the null distribution
x1 = ax.get_xbound()[0]; x2 = ax.get_xbound()[1]
nS = dfOutcome.groupby(Var).count()["LabelConfNull"]
nErrorSE = nErrorS * 1.96
# draw indeterminant region
plt.fill_between([x1,(x2+x1)/2],nMeanS[False] - nErrorSE[False],                  nMeanS[False] + nErrorSE[False], alpha = 0.2,color = "blue")
plt.fill_between([(x2+x1)/2,x2],nMeanS[True] - nErrorSE[True],                  nMeanS[True] + nErrorSE[True], alpha = 0.2,color = "blue")
# draw False side
plt.fill_between([x1,(x2+x1)/2],-0.05, nMeanS[False] - nErrorSE[False], alpha = 0.2, color="green")
plt.fill_between([x1,(x2+x1)/2],nMeanS[False] + nErrorSE[False],1.05, alpha = 0.2, color="red")
# draw True side
plt.fill_between([(x2+x1)/2,x2],-0.05, nMeanS[True] - nErrorSE[True], alpha = 0.2, color="green")
plt.fill_between([(x2+x1)/2,x2],nMeanS[True] + nErrorSE[True],1.05, alpha = 0.2, color="red")
plt.axhline(y = nMeanS[False], xmin = x1 ,xmax = (x2+x1)/2)
plt.axhline(y = nMeanS[True], xmin = (x2+x1)/2 ,xmax = x2)
plt.xlabel("Outcome")
sns.despine(top=True, right=True, left=True, bottom=True)
plt.ylabel("")
plt.xticks([0,1],["Poor","Good"])
ax.set_yticks([0,0.25,0.5,0.75,1])
ax.set_yticklabels(["100%","75%","50%","75%","100%"])
ax.set_ylabel("Confidence")
plt.ylim([-0.05,1.05])
plt.show()

#### Observe classification performance 

In[188]:

misclassifed people based on null based on combining the null distribution of outcomes

In [ ]:
dfOutcome["ResultTrue"] = 1
# those with positive outcome
po = dfOutcome[dfOutcome[Var] == True]["LabelConf"]
no = dfOutcome[dfOutcome[Var] == False]["LabelConf"]
dfOutcome["ResultTrue"].loc[dfOutcome[Var] == True] = po < (nMean - nCI)
dfOutcome["ResultTrue"].loc[dfOutcome[Var] == False] = no < (nMean - nCI)
# what were the actual predictions?
predtrue = np.ones((nSub,)) * np.nan
vpPred = dfOutcome["LabelConf"][dfOutcome[Var] == True] < (nMean - nCI)
vnPred= dfOutcome["LabelConf"][dfOutcome[Var] == False] > (nMean - nCI)
predtrue[dfOutcome[Var][(dfOutcome[Var] == True)].index] = vpPred
predtrue[dfOutcome[Var][(dfOutcome[Var] == False)].index] = vnPred

In[189]:

sort into true and false positives

In [ ]:
tp = np.sum(dfOutcome[dfOutcome[Var] == True]['ResultTrue'] == True);
tn = np.sum(dfOutcome[dfOutcome[Var] == False]['ResultTrue'] == True);
fp = np.sum(dfOutcome[dfOutcome[Var] == False]['ResultTrue'] == False)
fn = np.sum(dfOutcome[dfOutcome[Var] == True]['ResultTrue'] == False)
specificity = tn / (tn + fp)
sensitivity = tp / (tp + fn)
# print results
print("Sensitivity: ", sensitivity)
print("Specificity: ", specificity)
print("Individual Balanced Accuracy:", balanced_accuracy_score(dfOutcome[Var],predtrue))

In[190]:

misclassifed people based on null separated between outcomes

In [ ]:
dfOutcome["Result"] = 1
# those with positive outcome
po = dfOutcome[dfOutcome[Var] == True]["LabelConf"]
no = dfOutcome[dfOutcome[Var] == False]["LabelConf"]
dfOutcome["Result"].loc[dfOutcome[Var] == True] = po < (nMeanS[True] - nErrorSE[True])
dfOutcome["Result"].loc[dfOutcome[Var] == False] = no < (nMeanS[False] - nErrorSE[False])
# what were the actual predictions?
pred = np.ones((nSub,)) * np.nan
vpPred = dfOutcome["LabelConf"][dfOutcome[Var] == True] < (nMeanS[True] - nErrorSE[True])
vnPred= dfOutcome["LabelConf"][dfOutcome[Var] == False] > (nMeanS[False] - nErrorSE[False])
pred[dfOutcome[Var][(dfOutcome[Var] == True)].index] = vpPred
pred[dfOutcome[Var][(dfOutcome[Var] == False)].index] = vnPred

In[191]:

In [ ]:
tp = np.sum(dfOutcome[dfOutcome[Var] == True]['Result'] == True);
tn = np.sum(dfOutcome[dfOutcome[Var] == False]['Result'] == True);
fp = np.sum(dfOutcome[dfOutcome[Var] == False]['Result'] == False)
fn = np.sum(dfOutcome[dfOutcome[Var] == True]['Result'] == False)
specificity = tn / (tn + fp)
sensitivity = tp / (tp + fn)
print("Sensitivity: ", sensitivity)
print("Specificity: ", specificity)
print("Individual Balanced Accuracy:", balanced_accuracy_score(dfOutcome[Var],pred))

 Generally the results of these two appraoches are consistent, but it is unclear which is more appropriate.

 statistically evaluate group level results<br>
take average over each fold

In [ ]:
GroupBalancedAccuracy = np.mean(tROC,1)
GroupBalancedAccuracyMed = np.median(tROC,1)
# take average of null
GroupBalancedAccuracyNull = np.mean(tROCNull,1)

get parameters of the results

In [ ]:
Mu = np.mean(GroupBalancedAccuracy); Sigma = np.std(GroupBalancedAccuracy)
print("Group Balanced Accuracy",Mu)
NullMu = np.mean(GroupBalancedAccuracyNull); NullSigma = np.std(GroupBalancedAccuracyNull)
# get confidence interval
GroupCI = Sigma * 1.96
print("Group Balanced Accuracy Confidence Interval", "[", Mu - GroupCI, ",",Mu + GroupCI,"]")
# get significance 
ZGroup = (Mu - NullMu)/NullSigma
PGroup = norm.sf(ZGroup) 
print("Z Score Group",ZGroup)
print("p value Group", PGroup)
# #### Compute statistics on label confidence values 

In[193]:

or nulls separated between outcomes

In [ ]:
zval = np.ones((nSub,)) * np.nan
vt = dfOutcome["LabelConf"][dfOutcome[Var] == True                                               ].apply(lambda x: ((x - nMeanS[True])/nErrorS[True]))
vf = dfOutcome["LabelConf"][dfOutcome[Var] == False                                               ].apply(lambda x: ((x - nMeanS[False])/nErrorS[False]))
zval[dfOutcome[Var][(dfOutcome[Var] == True)].index] = vt
zval[dfOutcome[Var][(dfOutcome[Var] == False)].index] = vf
dfOutcome["z-value"] = zval

In[194]:

or nulls based on combining outcome

In [ ]:
zval = np.ones((nSub,)) * np.nan
vt = dfOutcome["LabelConf"][dfOutcome[Var] == True                                               ].apply(lambda x: ((x - nMean)/nError))
vf = dfOutcome["LabelConf"][dfOutcome[Var] == False                                               ].apply(lambda x: ((x - nMean)/nError))
zval[dfOutcome[Var][(dfOutcome[Var] == True)].index] = vt
zval[dfOutcome[Var][(dfOutcome[Var] == False)].index] = vf
dfOutcome["z-value-true"] = zval
dfOutcome["p-value-true"] = np.round(norm.sf(-dfOutcome["z-value-true"]),6)

print confidence scores for incorrect and correct people

In [ ]:
tConf = dfOutcome[dfOutcome["ResultTrue"] == 1]["LabelConf"]
tConf = 1 - tConf
fConf = dfOutcome[dfOutcome["ResultTrue"] == 0]["LabelConf"]
print("Mean Confidence Correct ",np.mean(tConf))
print("Mean Confidence Incorrect ",np.mean(fConf))

In [ ]:
print("STD Confidence Correct ",np.std(tConf))
print("STD Confidence Incorrect ",np.std(fConf))
# #### Plot confusion matrix

In[195]:

In [ ]:
from  matplotlib.colors import LinearSegmentedColormap
cmap=LinearSegmentedColormap.from_list('Custom',["red","white", "green"], N=123) 
groupProb = confusion_matrix(dfOutcome[Var],predtrue)
sns.set_theme(style = "white", rc={'figure.figsize': (3.1,3.1),"xtick.labelsize": 10,             'xtick.alignment':'center',"ytick.labelsize": 10,'font.family':'Arial','figure.dpi':500,             "axes.labelsize":12,"figure.facecolor":"white",'axes.facecolor':'white',"axes.spines.right": False,"axes.spines.left": False,                "axes.spines.top": False,"axes.spines.bottom": False})

In[196]:

In [ ]:
sns.heatmap(groupProb, annot = True,cmap = cmap,annot_kws={"fontsize":12},cbar = False,vmin=4, vmax=8,alpha = 0.2)
plt.xticks([0.5,1.5], labels = ["Good Outcome", "Poor Outcome"])
plt.yticks([0.5,1.5], labels = ["Good Outcome", "Poor Outcome"])
plt.xlabel("Actual Outcome")
plt.ylabel("Predicted Outcome")
plt.show()

#### Plot influential features<br><br>
Plot influential centroids after controlling for chance level differences between centroid (based on CentroidNull).<br>

In[197]:

Compute probability of centroid <br>
determine distribution of centroids

In [ ]:
CentroidsNull = np.mean(CentroidsNull,1) # average over folds
nCentroids = np.stack(CentroidsNull)
CentroidsReal = np.mean(CentroidsReal,1) # average over folds
Centroids = np.mean(CentroidsReal,0)
nullCentMean = np.mean(nCentroids[:,0,:] - nCentroids[:,1,:],axis = 0)
nullCentStd = np.std(nCentroids[:,0,:] - nCentroids[:,1,:],axis = 0)
# compute z score
aCentroid = Centroids[0,:] - Centroids[1,:]
zCentroid = (aCentroid - nullCentMean)/nullCentStd
# compute p value of z-score
from scipy.stats import norm
norm.sf(abs(zCentroid))*2
tnIdx = np.argsort(zCentroid)[0:3]
fig = plt.figure()
ax = plt.axes(projection ="3d")
c = Yout
x1 =  Xo[c,tnIdx[0]]; 
y1 = Xo[c,tnIdx[1]];
z1 = Xo[c,tnIdx[2]];
    # Create Plot
ax.scatter3D(x1,y1,z1,c = pal[-5], s = 40)
c = Yout == 0 
x2 =  Xo[c,tnIdx[0]];
y2 = Xo[c,tnIdx[1]];
z2 = Xo[c,tnIdx[2]];
    # Create Plot
#ax.scatter3D(x2,y2,z2,c="blue",s = dfOutcome["GCS_E"][c].values * 100)
ax.scatter3D(x2,y2,z2, c = pal[0],s = 40)
ax.set_xlabel("\n" + networkLabs[tnIdx[0]])
ax.set_ylabel("\n" + networkLabs[tnIdx[1]])
ax.set_zlabel("\n" + networkLabs[tnIdx[2]])
#ax.legend(["Good Outcome","Poor Outcome"],bbox_to_anchor=(1.5, 1))
#fig.suptitle("True Labels")
# Show plot
plt.show()

grab the networks with the largest difference between outcome but plot <br><br>
as a function of correct vs. incorrectly labelled groups

In[198]:

grab the networks with the largest difference between outcome and plot<br>
as a function of correct vs. incorrectly labelled groups

In [ ]:
Centroids = np.mean(CentroidsReal,0)
# compute z score
aCentroid = Centroids[0,:] - Centroids[1,:]
zCentroid = (aCentroid - nullCentMean)/nullCentStd
tnIdx = np.argsort(zCentroid)[0:3]
# print significance
print("Feature Importance", -zCentroid)
print("Feature Importance Significance", norm.sf(np.abs(zCentroid))*2)

In[199]:

In [ ]:
fig = plt.figure(figsize = (10, 7))
ax = plt.axes(projection ="3d")
# set cluster and values for label
c = (dfOutcome["ResultTrue"] == 0).values
x1 =  Xo[c,tnIdx[0]]; 
y1 = Xo[c,tnIdx[1]];
z1 = Xo[c,tnIdx[2]];
    # Create Plot
ax.scatter3D(x1,y1,z1,c="red")
c = (dfOutcome["ResultTrue"] == 1).values
x2 =  Xo[c,tnIdx[0]];
y2 = Xo[c,tnIdx[1]];
z2 = Xo[c,tnIdx[2]];
    # Create Plot
#ax.scatter3D(x2,y2,z2,c="blue",s = dfOutcome["GCS_E"][c].values * 100)
ax.scatter3D(x2,y2,z2,c="blue")
ax.set_xlabel("\n" + networkLabs[tnIdx[0]])
ax.set_ylabel("\n" + networkLabs[tnIdx[1]])
ax.set_zlabel("\n" + networkLabs[tnIdx[2]], rotation=60)
plt.legend(["Incorrectly Labelled","Correctly Labelled"])
#fig.suptitle("True Labels")
# Show plot
plt.show()

#### plot feature importance bar plots

In[200]:

In [ ]:
fNull = np.stack(CentroidsNull)
# null of difference between group 1 and 2
fNull = fNull[:,0,:] - fNull[:,1,:]

In[201]:

create CI

In [ ]:
CIh = np.mean(fNull) + (np.std(fNull) * 1.96)
CIl = np.mean(fNull) - (np.std(fNull) * 1.96)
centdf = pd.DataFrame([-aCentroid.T],columns = networkLabs)
centnulldf = pd.DataFrame(fNull,columns = networkLabs)
centnulldf = pd.melt(centnulldf,value_vars = networkLabs.tolist())
centnulldf.columns = ["Network","Null"]
centnulldf["Network"] = centnulldf["Network"].astype("category")
fig,ax = plt.subplots(1,1)
x = list(range(0,len(networkLabs)))
sns.barplot(x,-aCentroid,palette = "Paired")
ax.axhline(y = CIh, xmin = ax.get_xbound()[0], xmax = ax.get_xbound()[1],color = pal[-5]) 
ax.axhline(y = CIl, xmin = ax.get_xbound()[0], xmax = ax.get_xbound()[1],color = pal[0]) 
ax.set_ylim([-.4,.4])
# plot the confidence interval of the null distribution
ax.set_xticks(x)
ax.set_xticklabels(networkLabs,rotation=90)
plt.show()

In[202]:

lot as centroid difference

In [ ]:
fig,ax = plt.subplots(1,1)
x = list(range(0,len(networkLabs)))
sns.barplot(x,-aCentroid,palette = "Paired",ax = ax)
ax.set_xticks(x)
ax.set_xticklabels(networkLabs,rotation=90)
plt.show()

In[203]:

plot as z-score

In [ ]:
fig,ax = plt.subplots(1,1)
x = list(range(0,len(networkLabs)))
sns.barplot(x,-zCentroid,palette = "Paired",ax = ax)
# set significance bar
ax.axhline(y = 1.96,linestyle = '--',color = pal[9])
ax.set_xticks(x)
ax.set_xticklabels(networkLabs,rotation=90)
ax.set_ylabel("Predictor Importance (Z-Score)")
plt.show()

In[204]:

plot nulls themselves

In [ ]:
fig,ax = plt.subplots(1,1)
sns.barplot(data = centnulldf,x = "Network",y = "Null",palette = "Paired",ax = ax)
ax.set_xticks(x)
ax.set_xticklabels(networkLabs,rotation=90)
ax.set_ylabel("Average Centroid Difference")
plt.show()
# feature correlation

#### plot results to distinguish differences between correct and incorrect patients

In[205]:

first reorganized the results dataframe

In [ ]:
dfX = pd.DataFrame(X, columns = networkLabs)
dfX["Subject"] = np.arange(0,nSub).astype("int")
dfX = dfX.melt(id_vars = "Subject", value_vars = networkLabs)
dfM = pd.merge(left = dfOutcome.reset_index(), right = dfX,left_on = "level_0", right_on = "Subject")
dfM.rename(columns = {"variable" : "network"},inplace = True)
dfM["ResultTrue"] = dfM["ResultTrue"].apply(lambda x: "Correctly Labeled" if x else "Incorrectly Labeled")
dfM["ID-TF"] =  dfM["ResultTrue"]
dfM.loc[dfM["ResultTrue"] == "Incorrectly Labeled","ID-TF"] = \
    dfM.loc[dfM["ResultTrue"] == "Incorrectly Labeled","StudyID"]
dfM["ID-TF"] = dfM["ID-TF"].apply(lambda x: str("Patient " + str(round(x))) if type(x) == float else x)
sns.catplot(data = dfM, x = "network", y = "value", row = Var, hue = "ResultTrue", kind = "point")
plt.xticks(rotation = 65)

In[ ]:

In[209]:

also look at the incorrect predicted subjects more directly<br>
set plotting themes

In [ ]:
sns.set_theme(style = "white", rc={'figure.figsize': (4.5,3.1),"xtick.labelsize": 10,             'xtick.alignment':'center',"ytick.labelsize": 10,'font.family':'Arial','figure.dpi':600,"legend.fontsize":10,             "axes.labelsize":12,"figure.facecolor":"w","axes.spines.right": False,"axes.spines.left": False,                "axes.spines.top": False,"axes.spines.bottom": False})

In[212]:

In [ ]:
fig, ax = plt.subplots(nrows = 2,sharex = True,sharey = True)
sns.pointplot(data = dfM[dfM[Var] == 0], x = "network", y = "value", hue = "ID-TF", row = Var,ax = ax[0],              palette = [pal[0],pal[1],pal[2],pal[3]])
sns.pointplot(data = dfM[dfM[Var]], x = "network", y = "value", hue = "ID-TF", row = Var,ax = ax[1],              palette = [pal[-5],pal[-3],pal[-4]])
#fig.supylabel("Spatial Correlation")
#fig.supxlabel("Network")
ax[0].legend([],frameon=False); ax[1].legend([],frameon=False); 
ax[0].legend(bbox_to_anchor=(1.05, 1)); ax[1].legend(bbox_to_anchor=(1.05, 1));
ax[0].set_xlabel(''); ax[1].set_xlabel('')
ax[0].set_ylabel(''); ax[1].set_ylabel('')
plt.xticks(rotation = 90)
plt.show()

look at cross correlation between networks<br>
set plotting themes

In [ ]:
sns.set_theme(style = "white", rc={'figure.figsize': (3.14,3.1),"xtick.labelsize": 10,             'xtick.alignment':'center',"ytick.labelsize": 10,'font.family':'Arial','figure.dpi':600,"legend.fontsize":10,             "axes.labelsize":12,"figure.facecolor":"w","axes.spines.right": False,"axes.spines.left": False,                "axes.spines.top": False,"axes.spines.bottom": False})
fig, ax = plt.subplots()
sns.heatmap(np.corrcoef(Xo.T),cmap = sns.color_palette("vlag", as_cmap=True),vmin = -1, vmax = 1)
plt.xticks(np.arange(0,len(networkLabs)),networkLabs,rotation = 90)
plt.yticks(np.arange(0.5,len(networkLabs) + 0.5),networkLabs,rotation = 0)
plt.show()